In [ ]:
import os
import sys
os.chdir('/local/home/dhaziza/entrack')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="4"
sys.path.append('/local/home/dhaziza/entrack/')

import src.features as ft_def
import csv
import re

def intvals(s):
    vals = re.findall(r'\d+', s)
    return [int(i) for i in vals]

def intval(s):
    vals = intvals(s)
    return vals[0]

In [ ]:
# Koln data
csv_orig = 'data/raw/csv/orig/koln.csv'
csv_output = 'data/raw/csv/koln.csv'

data = []
with open(csv_orig) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append({
            ft_def.AGE: int(row['Alter']),
            ft_def.SEX: int(row['Geschlecht (male =1; female =2)'])-1,
            ft_def.STUDY_PATIENT_ID: intval(row['ID'])
        })

with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [ ]:
# ADNI AIBL
csv_orig = 'data/raw/csv/orig/adni_aibl.csv'
csv_output = 'data/raw/csv/adni_aibl.csv'

data = []
with open(csv_orig) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Imaging Protocol'] != 'Field Strength=1.5':
            continue
        # Reconstruct a single int subject id from something like '002_S_0295'
        subject_id = intvals(row['Subject ID'])
        subject_id = subject_id[0]*10000 + subject_id[1]
        data.append({
            ft_def.AGE: intval(row['Age']),
            ft_def.SEX: 0 if row['Sex'] == 'M' else 1,
            ft_def.STUDY_IMAGE_ID: int(row['Image ID']),
            ft_def.STUDY_PATIENT_ID: subject_id,
        })

num = 0.0
total_age = 0.0
with open(csv_output, "wb+") as csvfile:
    writer = csv.DictWriter(csvfile, data[0].keys(), dialect='excel')
    writer.writeheader()
    for row in data:
        writer.writerow(row)
        num += 1
        total_age += row['age']
print 'Avg age: %f' % (total_age/num)

In [ ]:
row